DeepLarning Couse HSE 2016 fall: Arseniy Ashuha, you can text me ```ars.ashuha@gmail.com```, ```https://vk.com/ars.ashuha``` 

<h1 align="center"> Image Captioning </h1> 

Тут мы проведем Вас через реализацию аннотирования картинок. Вам нужно помнить про глубокие и рекуррентные сети. 

Скачайте датасет, в нем уже построены признаки для изображений с помощью GoogleNet.

```bash
export dataset=https://s3.amazonaws.com/lasagne/recipes/datasets/coco_with_cnn_features.pkl
curl -o ./files/coco_with_cnn_features.pkl $dataset```

In [1]:
# Import all staff

import json
import pickle
import random
import theano
import lasagne
import sklearn
import numpy as np
import skimage.transform
import theano.tensor as T
import matplotlib.pyplot as plt
from utils.utils import * 

from time import time
from utils import googlenet
from lasagne import layers as ll
from collections import Counter
from lasagne.utils import floatX
from utils.utils import get_data_batch, prep_batch_for_network

%matplotlib inline

### Подготовка данных

In [ ]:
# Read Dataset
dataset = pickle.load(open('files/coco_with_cnn_features.pkl', 'rb'))

In [ ]:
dataset[0]['sentences']['tokens']

In [ ]:
# Build a Vocabulary
allwords = <Словарь слово:сколько раз встретилось> 

vocab  = ['#START#', '#END#']
vocab += [k for k, v in allwords.items() if v >= 5]

word_to_index = {w: i for i, w in enumerate(vocab)}
index_to_word = <Словарик индекс:слово>

In [ ]:
# Define of different constant 
EMBED = 128
BATCH_SIZE = 100
SEQUENCE_LENGTH = 10
CNN_FEATURE_SIZE = 1000
MAX_SENTENCE_LENGTH = SEQUENCE_LENGTH - 3 # 1 for image, 1 for start token, 1 for end token

### Ваша модель

In [ ]:
# Input Variable
x_snt = T.imatrix() # batch_size x words_ids 
x_img = <что-то с признаками из изображений> 
y_snt = <правильные слова>
mask  = <Маска чтобы предсказывать короткие последовательности>

In [ ]:
# Input Layer for words 
embed_snt = ll.InputLayer((None, SEQUENCE_LENGTH - 1), x_snt)
embed_snt = <Слой с кодированием каждого слова вектором> 

In [ ]:
# Input Layer for image features
embed_img = ll.InputLayer((None, CNN_FEATURE_SIZE), x_img)
embed_img = <Слой преобразующий признаки картинки к размерности = размерности вектора для каждого слова>
embed_img = ll.ReshapeLayer(embed_img, ([0], 1, [1]))

In [ ]:
# Concatinate image features and word embedings in one sequence 
rnn_in = <Соедините embed_img и embed_snt в одну последовательность>

In [ ]:
# Make a recurent part 
rnn_in = <Добавьте немного регуляризации>
rnn = <Теперь нужно рекурентную часть сделать>
rnn_out = <Добавьте немного регуляризации>

In [ ]:
# Decoding of rnn hiden states
rnn_out = <Нужно сделать Reshape чтобы все хорошо можно было декодировать>
decoder = <Декодирование через DenseLayer>
decoder = <Обратный Reshape> # <--- Тут уже должен быть тензор (батч, длина последовательности, размер словаря)

In [ ]:
def calc_cross_ent(net_output, mask, targets):
    preds, targets = T.reshape(net_output, (-1, len(vocab))), T.flatten(targets)
    # Сумма только по правильной длине
    cost = T.nnet.categorical_crossentropy(preds, targets)[T.flatten(mask).nonzero()]
    return cost

output = ll.get_output(decoder, deterministic=False)
loss = T.mean(calc_cross_ent(output, mask, y_snt))
all_params = ll.get_all_params(decoder)
updates = <Метод оптимизации>

In [ ]:
# (x_cnn, x_sentence, mask, y_sentence)
f_train = <Скомпилируйте функцию для обучения>
f_val   = <Скомпилируйте функцию для обучения>

In [ ]:
loss_train, norm, st = 0, 0, time()
for iteration in range(0, 5000):
    print '.',
    batch = prep_batch_for_network(
        get_data_batch(dataset, BATCH_SIZE, MAX_SENTENCE_LENGTH), SEQUENCE_LENGTH, word_to_index)
    x_cnn, x_sentence, y_sentence, mask = batch
    loss_train += f_train(x_cnn, x_sentence, mask, y_sentence)
    if not iteration % 40:
        print int(time() - st), 'sec.',
        st = time()
        
        batch = get_data_batch(dataset, BATCH_SIZE, MAX_SENTENCE_LENGTH, split='val')
        x_cnn, x_sentence, y_sentence, mask = prep_batch_for_network(batch, SEQUENCE_LENGTH, word_to_index)
        loss_val = f_val(x_cnn, x_sentence, mask, y_sentence)
        print('\nIt: {}, loss_train: {}, val loss: {}'.format(iteration, loss_train/40, loss_val))
        loss_train, norm = 0, 0
        
print '\nFinish =)'

### Применение модели

In [ ]:
# Загрузим сеть GoogLeNet
cnn_layers = googlenet.build_model()
cnn_input_var = cnn_layers['input'].input_var
cnn_feature_layer = cnn_layers['loss3/classifier']
cnn_output_layer = cnn_layers['prob']

get_cnn_features = theano.function([cnn_input_var], lasagne.layers.get_output(cnn_feature_layer))

gnw = pickle.load(open('files/blvc_googlenet.pkl'))
CLASSES = np.array(gnw['synset words'])
lasagne.layers.set_all_param_values(cnn_output_layer, gnw['param values'])

In [ ]:
im = plt.imread('./files/Dog-and-Cat.jpg')
rawim, cnn_im = prep_image(im)

In [ ]:
plt.subplot(121); plt.imshow(im);
plt.subplot(122); plt.imshow(rawim);

In [ ]:
# Выведим самые вероятные классы
p = get_cnn_features(cnn_im)
max_prob_idx = p.argsort()[0][-5:]

for i in reversed(max_prob_idx):
    print CLASSES[i], p[0][i]

In [ ]:
# Получим функцию для генепации следующего слова
output = ll.get_output(decoder, deterministic=False)
f = theano.function([x_img, x_snt], output)

In [ ]:
def predict(x_cnn):
    x_sentence = np.zeros((1, SEQUENCE_LENGTH - 1), dtype='int32')
    words = []
    i = 0
    while True:
        i += 1
        p0 = f(x_cnn, x_sentence)
        pa = p0.argmax(-1)
        tok = pa[0][i]
        word = index_to_word[tok]
        if word == '#END#' or i >= SEQUENCE_LENGTH - 1:
            return ' '.join(words)
        else:
            x_sentence[0][i] = tok
            if word != '#START#':
                words.append(word)

In [ ]:
x_cnn = get_cnn_features(cnn_im)

In [ ]:
# Sample some predictions
for _ in range(10):
    print(predict(x_cnn))